# Full Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* Model: DistilBERT - I chose DistilBERT because it is a lightweight model that runs quickly and does not require changing all parameters. 
* Evaluation approach: For evaluation, I compared the accuracy of the model trainer to the accuracy of a fully fine-tuned model. 
* Fine-tuning dataset: https://huggingface.co/datasets/victor/real-or-fake-fake-jobposting-prediction

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
pip install peft transfomers


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import modules and packages
import torch 

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load Real or Fake Jobposting Dataset
# URL: https://huggingface.co/datasets/victor/real-or-fake-fake-jobposting-prediction

from datasets import load_dataset

# Split dataset into training and testing datasets
# Use seed as a repeatable constant for Shuffle 
# Use 2.5% of data for tests
# Attribution: https://huggingface.co/docs/datasets/en/process

dataset = load_dataset("victor/real-or-fake-fake-jobposting-prediction", 
    split="train").train_test_split(test_size=0.025, shuffle=True, seed=17)

# Split datasets into train (full dataset) and test (2.5% of dataset)
split_datasets = ["train", "test"]

# For partial fine-tuning 
# Use first 500 in training dataset
#dataset["train"] = dataset["train"].select(range(500))

# View Datasets
print(dataset)

# Load Tokenizer from Pre-trained distilbert model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Create tokenizer function 
# Attribution: Udacity "Adapting Foundation Models"
def tokenizer_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation = True)


# Tokenize Train and Test Datasets 
tokenize_datasets = {}
for set in split_datasets:
    tokenize_datasets[set] = dataset[set].map(tokenizer_function, batched=True)

# Rename columns for text classification
# Attribution: https://stackoverflow.com/questions/73290491/the-model-did-not-return-a-loss-from-the-inputs-labse-error
tokenize_datasets["train"] = tokenize_datasets["train"].rename_column("description", "text")
tokenize_datasets["train"] = tokenize_datasets["train"].rename_column("fraudulent", "labels")
tokenize_datasets["test"] = tokenize_datasets["test"].rename_column("description", "text")
tokenize_datasets["test"] = tokenize_datasets["test"].rename_column("fraudulent", "labels")

    
# Convert Model Predictions to Strings / Convert Labels to Integers
# Attribution: Udacity "Adapting Foundation Models"
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    # Create Real and Fake Labels
    num_labels=2,
    id2label={0: "REAL", 1: "FAKE"},  
    label2id={"REAL": 0, "FAKE": 1},
)

# Unfreeze Parameters of Base Model
# Attribution: https://huggingface.co/transformers/v4.2.2/training.html
for param in model.base_model.parameters():
    param.requires_grad = False

    
print(tokenize_datasets["train"].column_names)
print(tokenize_datasets["test"].column_names)

Generating train split:   0%|          | 0/17880 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'fraudulent'],
        num_rows: 17433
    })
    test: Dataset({
        features: ['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'fraudulent'],
        num_rows: 447
    })
})


Map:   0%|          | 0/17433 [00:00<?, ? examples/s]

Map:   0%|          | 0/447 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'text', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'labels', 'input_ids', 'attention_mask']
['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'text', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'labels', 'input_ids', 'attention_mask']


In [3]:
# View model 
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [4]:
# View Training Datasets
print("Training Dataset:\n")
print(tokenize_datasets["train"])

print("\n--------------------------------------------------\n")

print("Testing Dataset:\n")
# View Test Datasets
print(tokenize_datasets["test"])



Training Dataset:

Dataset({
    features: ['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'text', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 17433
})

--------------------------------------------------

Testing Dataset:

Dataset({
    features: ['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'text', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 447
})


In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy 


# Compute accuracy of model 
# Attribution: https://huggingface.co/docs/transformers/main_classes/trainer
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = numpy.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Configure Model Trainer 
# Attribution: https://huggingface.co/docs/transformers/main_classes/trainer

trainer_model = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/job_results_model",
        # Overwrite saved data each time code runs
        overwrite_output_dir=True,
        learning_rate=2e-2,
        per_device_train_batch_size=5,
        per_device_eval_batch_size=5,
        # Run a training loop w/ each epoch 
        evaluation_strategy="epoch",
        # Save the trained model
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
    ),
    train_dataset=tokenize_datasets["train"],
    eval_dataset=tokenize_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train the model
trainer_model.train()

# Evaluate the model
model_eval = trainer_model.evaluate()

/home/student/.local/lib/python3.10/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.194300,0.197606,0.941834


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
# Unfreeze model parameters prior to fine-tuning
# Attribution: Udacity "Adapting Foundation Models"
for param in model.parameters():
    param.requires_grad = True

# Configure PEFT Trainer
# Changes: Decrease learning rate, increase batch size, increase number of epochs
trainer_full_tuned = Trainer(
    model=model,
    args=TrainingArguments(
        # Create job_results_peft directory
        # Output training data to created directory
        output_dir="./data/job_results_full_fine_tuned",
        # Overwrite saved data each time code runs
        overwrite_output_dir=True,
        # Decrease learning rate to allow gradual fine-tuning 
        # Increase liklihood of accurate predictions 
        learning_rate=2e-5,
        # Increase batch size to provide more accurate prediction
        per_device_train_batch_size=12,
        per_device_eval_batch_size=12,
        # Run a training loop w/ each epoch 
        evaluation_strategy="epoch",
        # Save the trained model at each epoch
        save_strategy="epoch",
        # Increase epochs to allow more iterations for training
        num_train_epochs=2,
        weight_decay=0.01,
    ),
    train_dataset=tokenize_datasets["train"],
    eval_dataset=tokenize_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train the PEFT
trainer_full_tuned.train()

# Evaluate the PEFT
full_tuned_eval = trainer_full_tuned.evaluate()

/home/student/.local/lib/python3.10/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.164800,0.166906,0.953020
2,0.124500,0.152898,0.964206


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [7]:
# Compare results of Model and PEFT

print("Base Model Results: ")
# Evaluate the model
print(model_eval) 

# Access Eval Accuracy from Model Evaluation 
model_accuracy = model_eval['eval_accuracy']

print("\n")

print("Fine-Tuned Model Results: ")
# Evaluate the PEFT
print(full_tuned_eval)

# Access Eval Accuracy from PEFT Evaluation 
full_tuned_accuracy = full_tuned_eval['eval_accuracy']


Base Model Results: 
{'eval_loss': 0.1976059377193451, 'eval_accuracy': 0.941834451901566, 'eval_runtime': 7.2759, 'eval_samples_per_second': 61.436, 'eval_steps_per_second': 12.37, 'epoch': 1.0}


Fine-Tuned Model Results: 
{'eval_loss': 0.15289801359176636, 'eval_accuracy': 0.9642058165548099, 'eval_runtime': 8.0982, 'eval_samples_per_second': 55.197, 'eval_steps_per_second': 4.692, 'epoch': 2.0}


In [8]:
if full_tuned_accuracy > model_accuracy:
    tuned_improvement = (full_tuned_accuracy - model_accuracy) * 100
    print('Full Fine-Tuning improved prediction accuracy by {:.2f}%!'.format(tuned_improvement))
else: 
    print("No improvement with fine-tuning. Additional training is recommended.")

Full Fine-Tuning improved prediction accuracy by 2.24%!
